In [1]:
import pandas as pd
import config_local.local_config as local_config

In [2]:
train = pd.read_csv(local_config.TRAIN_PROCESS3_CSV)  
test = pd.read_csv(local_config.TEST_PROCESS3_CSV)

In [3]:
to_drop = []#['Street']
train.drop(columns=to_drop, inplace=True, axis=1)

In [4]:
for df in (train, test):
    df["Age"] = df["YrSold"] - df["YearBuilt"]
    df["RemodAge"] = df["YrSold"] - df["YearRemodAdd"]

In [5]:
train.to_csv(__import__("pathlib").Path(local_config.TRAIN_PROCESS4_CSV).resolve().parent / "train_process4.csv", index=False)  
test.to_csv(__import__("pathlib").Path(local_config.TEST_PROCESS4_CSV).resolve().parent / "test_process4.csv", index=False) 